In [26]:
import pandas as pd
import sqlalchemy as sa
import sqlite3
import datetime as dt

# Create A SQLite Database By Connecting to It With SQLAlchemy

In [27]:
# Create a SQLite Database in the local directory
engine = dbEngine = sa.create_engine('sqlite://///Users/dalbrecht/PycharmProjects/ByJove/notebooks/lab.db')

## Lets create a table and Populate It

In [28]:
# Create Table "Consoles"

consoles_create = """
CREATE TABLE CONSOLES (
   console_id INTEGER PRIMARY KEY AUTOINCREMENT,
   console_name VARCHAR,
   release_date VARCHAR
)"""

engine.execute(consoles_create)

In [29]:

basic_consoles_insert = """
INSERT INTO CONSOLES (console_name, release_date)
       VALUES ("N64", "1996-09-29 00:00:00") """

engine.execute(basic_consoles_insert)

In [30]:

# Insert Some Records
records = [
    {
        "console_name": "WII",
        "release_date": dt.datetime(2006, 11, 19)
    },
    {
        "console_name": "NES",
        "release_date": dt.datetime(1985, 10, 18)
    },
    {
        "console_name": "SNES",
        "release_date": dt.datetime(1990, 11, 21)},
    {
        "console_name": "Switch",
        "release_date": dt.datetime(1917, 3, 3)
    },
    {
        "console_name": "COOLBOX 2050",
        "release_date": dt.datetime(2050, 1, 1)
    }]

# This is a "bound" insert statement

bound_insert = """
INSERT INTO CONSOLES (console_name, release_date)
       VALUES (?,?) """

list_of_records = [(x["console_name"], x["release_date"].strftime("%Y-%m-%d %H:%M:%S")) for x in records]

engine.execute(bound_insert, list_of_records)

# Querying Example

In [31]:
with engine.connect() as connection:
    result = connection.execute("SELECT * FROM CONSOLES")
    for row in result:
        row_date = dt.datetime.strptime(row['release_date'], "%Y-%m-%d %H:%M:%S")
        date_str = row_date.strftime("%d %b, %Y")
        print(f"{row['console_id']}, {row['console_name']}, {date_str}")

1, N64, 29 Sep, 1996
2, WII, 19 Nov, 2006
3, NES, 18 Oct, 1985
4, SNES, 21 Nov, 1990
5, Switch, 03 Mar, 1917
6, COOLBOX 2050, 01 Jan, 2050


## Let's Fix A Record

In [32]:
update_query = """
UPDATE CONSOLES
SET release_date = "2017-03-03 00:00:00"
WHERE console_name = "Switch"
"""
connection.execute(update_query)

ResourceClosedError: This Connection is closed

In [33]:
with engine.connect() as connection:
    connection.execute(update_query)

## Let's Delete A Record

In [34]:
delete_query = """
DELETE FROM CONSOLES
WHERE name = ?
"""
with engine.connect() as connection:
    connection.execute(delete_query, ["COOLBOX 2050"])

OperationalError: (sqlite3.OperationalError) no such column: name
[SQL: 
DELETE FROM CONSOLES
WHERE name = ?
]
[parameters: ('COOLBOX 2050',)]
(Background on this error at: https://sqlalche.me/e/14/e3q8)

In [35]:
delete_query = """
DELETE FROM CONSOLES
WHERE console_name = ?
"""
with engine.connect() as connection:
    connection.execute(delete_query, ["COOLBOX 2050"])

# INSPECT IN DBEAVER

##Load Into Pandas

In [36]:

dataframe = pd.read_sql_table("consoles", engine)
dataframe

,console_id,console_name,release_date
0,1,N64,1996-09-29 00:00:00
1,2,WII,2006-11-19 00:00:00
2,3,NES,1985-10-18 00:00:00
3,4,SNES,1990-11-21 00:00:00
4,5,Switch,2017-03-03 00:00:00


In [37]:
dataframe.dtypes

console_id       int64
console_name    object
release_date    object
dtype: object

In [38]:
dataframe["release_date"] = pd.to_datetime(dataframe["release_date"])
dataframe

,console_id,console_name,release_date
0,1,N64,1996-09-29
1,2,WII,2006-11-19
2,3,NES,1985-10-18
3,4,SNES,1990-11-21
4,5,Switch,2017-03-03


In [39]:
dataframe.dtypes

console_id               int64
console_name            object
release_date    datetime64[ns]
dtype: object

In [40]:
large_dataset = pd.read_csv("vgsales.csv")
large_dataset

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37
...,...,...,...,...,...,...,...,...,...,...,...
16593,16596,Woody Woodpecker in Crazy Castle 5,GBA,2002.0,Platform,Kemco,0.01,0.00,0.00,0.00,0.01
16594,16597,Men in Black II: Alien Escape,GC,2003.0,Shooter,Infogrames,0.01,0.00,0.00,0.00,0.01
16595,16598,SCORE International Baja 1000: The Official Game,PS2,2008.0,Racing,Activision,0.00,0.00,0.00,0.00,0.01
16596,16599,Know How 2,DS,2010.0,Puzzle,7G//AMES,0.00,0.01,0.00,0.00,0.01


In [41]:
large_dataset.dtypes

Rank              int64
Name             object
Platform         object
Year            float64
Genre            object
Publisher        object
NA_Sales        float64
EU_Sales        float64
JP_Sales        float64
Other_Sales     float64
Global_Sales    float64
dtype: object

In [42]:
large_dataset.to_sql("VIDEO_GAME_SALES", engine)

16598